## Preprocessing, following Neural Network Process Flow. Identify categorical data columns with > 10 unique values, under which bucket data columns with extreme rare number of values into "other" column.

In [252]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [253]:
# Check data types in DataFrame
application_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [254]:
# Feature(s) and target(s) in this dataset
print(f'All columns other than EIN and NAME which are undesired variables and IS_SUCCESSFUL are features.')
print(f'IS_SUCCESFFUL column is the target.')

All columns other than EIN and NAME which are undesired variables and IS_SUCCESSFUL are features.
IS_SUCCESFFUL column is the target.


In [255]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df_drop = application_df.drop(columns = ['EIN','NAME'])
df_drop

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [256]:
# from IPython.core import application
# Generate our categorical variable lists
attrition_cat = df_drop.dtypes[df_drop.dtypes == "object"].index.tolist()
attrition_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [257]:
# Determine the number of unique values in each column to identify columns with > 10 unique values.
df_drop.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [258]:
# Look at APPLICATION_TYPE value counts for binningm, and make it a data frame.
df_drop['APPLICATION_TYPE'].value_counts()
df_application_type = pd.DataFrame(df_drop['APPLICATION_TYPE'].value_counts(),df_drop['APPLICATION_TYPE'].value_counts().index)
df_application_type

,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156
T13,66


In [259]:
# Choose cutoff value = 528 and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = df_application_type[df_application_type['APPLICATION_TYPE']<528].index.values

# Replace in dataframe
for app in application_types_to_replace:
    df_drop['APPLICATION_TYPE'] = df_drop['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df_drop['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [260]:
# Look at CLASSIFICATION value counts for binning
df_drop['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [261]:
# You may find it helpful to look at CLASSIFICATION value counts > 1. First to create data frame and then choose value counts > 1.
df_classification = pd.DataFrame(df_drop['CLASSIFICATION'].value_counts(), df_drop['CLASSIFICATION'].value_counts().index)
df_classification[df_classification['CLASSIFICATION']>1]

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114


In [262]:
# cutoff value = 200 and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = df_classification[df_classification['CLASSIFICATION'] < 200].index.values
# classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    df_drop['CLASSIFICATION'] = df_drop['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
df_drop['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [263]:
# Check values in ASK_AMT column
df_ask_amt = pd.DataFrame(df_drop['ASK_AMT'].value_counts(), df_drop['ASK_AMT'].value_counts().index)
df_ask_amt

,ASK_AMT
5000,25398
10478,3
15583,3
63981,3
6725,3
...,...
5371754,1
30060,1
43091152,1
18683,1


In [264]:
# cutoff value is a single digit and create a list of ask_amt_to_replace
# use the variable name `ask_amt_to_replace`
ask_amt_to_replace = df_ask_amt[df_ask_amt['ASK_AMT'] < 10].index.values
ask_amt_to_replace

# Replace in dataframe
for cls in ask_amt_to_replace:
    df_drop['ASK_AMT'] = df_drop['ASK_AMT'].replace(cls,"Other")

# Check to make sure binning was successful
df_drop['ASK_AMT'].value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [265]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(df_drop)

In [266]:
# Split our preprocessed data into our features(X) and target(y) arrays
y = df_drop.IS_SUCCESSFUL.values
X = df_drop.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [267]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: ignored

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=14))  ## input layer
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))                ## first hidden layer
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))                ## second hidden layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))              ## output layer

# First hidden layer
#  YOUR CODE GOES HERE

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/100
25724/25724 [==============================] - 2s 80us/sample - loss: 0.5700 - acc: 0.7234
Epoch 2/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5556 - acc: 0.7294
Epoch 3/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5529 - acc: 0.7304
Epoch 4/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5514 - acc: 0.7318
Epoch 5/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5495 - acc: 0.7322
Epoch 6/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5488 - acc: 0.7330
Epoch 7/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5482 - acc: 0.7335
Epoch 8/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5475 - acc: 0.7326
Epoch 9/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5468 - acc: 0.73491s - lo
Epoch 10/100
25724/25724 [==============================] - 1s 49u

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391


In [ ]:
# Export our model to HDF5 file
